In [93]:
import numpy as np
import time
from timeit import timeit as timeit
import subprocess
import os

In [94]:
def SNPtoSMC(dist, SNP, POS):
    """
    dist : distinguished ind
    SNP : snp matrix
    POS : positions matrix of SNP
    """
    N = SNP.shape[0]
    N = N if N%2 == 0 else N - 1
    assert dist < N/2
    
    dist *= 2
    M = POS.shape[0]
    SMC = np.zeros((2*M,4), dtype=int)
    SMC[:,-1] = N*np.ones(2*M, dtype=int) - 2
    SMC[:, 0] = np.ones(2*M, dtype=int)
    d = SNP[dist, :] + SNP[dist+1, :]
    u = np.sum(SNP, axis=0, dtype=int) - d
    SMC[1::2, 1:-1] = np.transpose([d, u]) 
    SMC[0::2, 0:-1] = np.transpose([POS - 1, np.zeros(M, dtype=int), np.zeros(M, dtype=int)])
    todelete = np.argwhere(POS <= 1).transpose()*2
    return np.delete(SMC, todelete, 0)

In [95]:
def deleteZerosSNP(SNP, POS):
    ids = POS == 0
    return np.delete(SNP, ids, 1), np.delete(POS, ids)

def loadfile(scenario, i):
    file_name = "hdgp/scenario_" + scenario +"/hdgp_" + scenario + "_" + str(i) + ".npz"
    data = np.load(file_name)
    return data

In [96]:
def header_smc(pid, dist, n_ind):
    """
    pid: id de la population considérée
    dist: individu distingué
    n_ind: nombre d'individus
    """
    header = 'SMC++ {"version": "1.15.4.dev18+gca077da", "pids": ["'+ pid +'"], "undist": [['
    for i in range(n_ind):
        if(i != dist):
            indid = pid + str(i)
            header += '["' + indid + '", 0], ["' + indid + '", 1], '
    indid = pid + str(dist)
    header = header[:-2] + ']], "dist": [[["' + indid +'", 0], ["' + indid + '", 1]]]}'
    return header

In [97]:
## test sur un exemple simple
def test():
    snp = np.array([
    [0, 0, 1, 1],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    ])
    pos = np.array([3, 5, 1, 10])
    print(SNPtoSMC(0, snp, pos))
    print(SNPtoSMC(1, snp, pos))
    print(SNPtoSMC(2, snp, pos))
test()

[[2 0 0 4]
 [1 0 2 4]
 [4 0 0 4]
 [1 0 1 4]
 [1 1 0 4]
 [9 0 0 4]
 [1 2 1 4]]
[[2 0 0 4]
 [1 2 0 4]
 [4 0 0 4]
 [1 0 1 4]
 [1 0 1 4]
 [9 0 0 4]
 [1 0 3 4]]
[[2 0 0 4]
 [1 0 2 4]
 [4 0 0 4]
 [1 1 0 4]
 [1 0 1 4]
 [9 0 0 4]
 [1 1 2 4]]


In [98]:
## mesure du temps prise par la fonction SNPtoSMC
data = loadfile(scenario="11687", i="3")
n = 1000
SNP, POS = deleteZerosSNP(data['SNP'], data['POS'])
SMC = SNPtoSMC(0, data['SNP'], data['POS'])
print(SMC[:10])
time = timeit("SNPtoSMC(0, data['SNP'], data['POS'])", globals=globals(), number=n)/n
print(f"\nSNPtoSMC exec time {time:.5f}s")

assert SMC[:, 0].any() > 0

[[13774     0     0    10]
 [    1     1     0    10]
 [29507     0     0    10]
 [    1     0     1    10]
 [50438     0     0    10]
 [    1     1     8    10]
 [ 6364     0     0    10]
 [    1     1     0    10]
 [29786     0     0    10]
 [    1     1     7    10]]

SNPtoSMC exec time 0.00066s


In [99]:
def boucle():
    for d in os.listdir("hdgp"):
        for f in os.listdir("hdgp/" + d):
            data = np.load("hdgp/" + d + "/" + f)
            SNP, POS = deleteZerosSNP(data['SNP'], data['POS'])
            N = data['SNP'].shape[0]
            N = N if N%2 == 0 else N - 1
            for dist in range(0, int(N/2)):
                SMC = SNPtoSMC(dist, SNP, POS)
                filename = "out/" + d + "/" + f + str(dist) + ".smc"
                header = header_smc("hdgp", dist, N)
                np.savetxt(filename , SMC, delimiter=' ', fmt = "%d", header=header)
                
def one_scenario(d="11687"):
    d = f"scenario_{d}"
    for f in os.listdir("hdgp/" + d):
        data = np.load("hdgp/" + d + "/" + f)
        SNP, POS = deleteZerosSNP(data['SNP'], data['POS'])
        N = data['SNP'].shape[0]
        N = N if N%2 == 0 else N - 1
        for dist in range(0, int(N/2)):
            SMC = SNPtoSMC(dist, SNP, POS)
            filename = "out/" + d + "/" + f + str(dist) + ".smc"
            header = header_smc("hdgp", dist, N)
            np.savetxt(filename , SMC, delimiter=' ', fmt = "%d", header=header)

In [100]:
n = 3
time = timeit("one_scenario()", globals=globals(), number=n)/n
print(f"{time:.2f}s pour convertir tous les fichiers du scénario")

n = 3
time = timeit("one_scenario(d='16844')", globals=globals(), number=n)/n
print(f"{time:.2f}s pour convertir tous les fichiers du scénario")

1.12s pour convertir tous les fichiers du scénario
1.12s pour convertir tous les fichiers du scénario


In [92]:
subprocess.check_output("smc++ estimate .5e-9 --knots 22 --timepoints 0 34483 out/scenario_11687/*.smc", shell=True)

KeyboardInterrupt: 

In [101]:
subprocess.check_output("smc++ estimate .5e-9 --knots 22 --timepoints 0 34483 out/scenario_16844/*.smc", shell=True)

b'RUNNING THE L-BFGS-B CODE\n\n           * * *\n\nMachine precision = 2.220D-16\n N =           11     M =           10\n\nAt X0         0 variables are exactly at the bounds\n\nAt iterate    0    f=  1.16985D+09    |proj g|=  3.00000D+00\n\nAt iterate    1    f=  1.16981D+09    |proj g|=  3.66800D+00\n\nAt iterate    2    f=  1.16980D+09    |proj g|=  3.83224D+00\n\nAt iterate    3    f=  1.16980D+09    |proj g|=  3.87279D+00\n\nAt iterate    4    f=  1.16980D+09    |proj g|=  3.89727D+00\n\nAt iterate    5    f=  1.16980D+09    |proj g|=  4.23485D+00\n\nAt iterate    6    f=  1.16980D+09    |proj g|=  4.30274D+00\n\nAt iterate    7    f=  1.16980D+09    |proj g|=  4.42938D+00\n\nAt iterate    8    f=  1.16980D+09    |proj g|=  4.49623D+00\n\nAt iterate    9    f=  1.16980D+09    |proj g|=  4.50274D+00\n\nAt iterate   10    f=  1.16980D+09    |proj g|=  4.52009D+00\n\nAt iterate   11    f=  1.16980D+09    |proj g|=  4.47547D+00\n\nAt iterate   12    f=  1.16980D+09    |proj g|=  4.54